In [1]:
# bibliotecas
import math, random
import matplotlib
import decimal
import pandas as pd

In [2]:
# lendo os arquivos de entrada
initial_weights = open('initial_weights.txt','r+')

def read_dataset(path_file):
    '''função responsável em ler o arquivo de dataset'''
    dataset = list()
    file = open(path_file, 'r+')
    for lines in file:
        values_line = list()
        
        lines = lines.rstrip().split('; ')
        for line in lines:
            if ', ' in line:
                dataset.append(line.rstrip().split(', '))
            else:
                dataset(line)
    return dataset
    
dataset = read_dataset("dataset.txt")
print(dataset)
    
def read_def_network(path_file):
    '''função responsável em ler o arquivo de definição da rede neural'''
    file = open(path_file, "r+")
    
    network = list()
    for line in file:
        network.append(line.rstrip())
    return network

def_network = read_def_network('network.txt')

[['0.5', '0.7'], ['1.0', '0.0']]


#### Rede neural composta por um número ajustável de neurônios e camadas e treinada via backpropagation

In [3]:
def create_network(initial_weights, n_hidden, n_output):
    '''função responsável em criar a rede neural a partir do arquivo de definição'''
    '''n_hidden pesos dos neurônios ocultos'''
    '''n_output pesos dos neurônios de saída'''
    i = 0
    network = list()
    lines = initial_weights.readlines()
    while i < n_hidden:
        layers = list()
        for layer in [line.split(', ') for line in lines[i].rstrip().split('; ')]:
            neurons = list()
            for neuron in layer:
                neurons.append(float(neuron))
            layers.append(neurons)
        network.append(layers)
        i += 1
    while i < n_hidden + n_output:
        layers = list()
        for layer in [line.split(', ') for line in lines[i].rstrip().split('; ')]:
            neurons = list()
            for neuron in layer:
                neurons.append(float(neuron))
            layers.append(neurons)
        network.append(layers)
        i += 1
    return network

def dot(v, w):
    '''função responsável em realizar a soma pondera das entradas'''
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def sigmoid(t):
    '''função responsável sobre a transferência do neurônio ativado'''
    return 1 / (1 + math.exp(-t))

def neuron_output(weights, inputs):
    return sigmoid(dot(weights, inputs))

def feed_forward(neural_network, input_vector):
    """takes in a neural network (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""
    outputs = []

    for layer in neural_network:
        input_with_bias = input_vector + [1]             # add a bias input
        output = [neuron_output(neuron, input_with_bias) # compute the output
                  for neuron in layer]                   # for this layer
        outputs.append(output)                           # and remember it

        # the input to the next layer is the output of this one
        input_vector = output
    return outputs

def backpropagate(network, input_vector, target):
    hidden_outputs, outputs = feed_forward(network, input_vector) # TODO: deve ficar mais generico para n camadas
    
    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target[i])
                     for i, output in enumerate(outputs)]
                     
    # adjust weights for output layer (network[-1])
    for i, output_neuron in enumerate(network[-1]):
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            output_neuron[j] -= output_deltas[i] * hidden_output

    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) * 
                      dot(output_deltas, [n[i] for n in network[-1]]) 
                     for i, hidden_output in enumerate(hidden_outputs)]

    # adjust weights for hidden layer (network[0])
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input

# construindo a rede neural
network = create_network(initial_weights, 1, 1)  # TODO: mudar para o arquivo def a rede

            
# treinamento
backpropagate(network, [float(dataset[0][0])], [float(dataset[0][1])]) # TODO: desenvolver para qualquer dataset


def predict(input):
    return feed_forward(network, input)[-1]

print(predict([0.42000]))

[0.7794307159924072]


#### Funcionalidade que permita, via linha de comando, informar a sua implementação a estrutura de uma rede de teste (i.e., estrutura de camadas/neurônios, pesos iniciais, e fator de regularização), e um conjunto de treinamento, e que retorne o gradiente calculado para cada peso

#### Funcionalidade que permita, via linha de comando, efetuar a verificação numérica do gradiente, a fim de checar a corretude da implementação de cada grupo

#### Um mecanismo para normalização das features/dados de treinamentos

#### Mecanismo para uso de regularização